In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os

# Custom libraries
from Components.TrainModel import DataModule#, TEMPUS
from Components.TickerData import TickerData
from Components.BackTesting import BackTesting

# Torch ML libraries
import torch
import torch.nn as nn
from torch.optim import AdamW

#device = "mps" if torch.backends.mps.is_available() else "cpu"
device = "cpu"

In [ ]:
#TODO: Feature importance with SHAP values and plot
#TODO: hyperparameter tuning
#TODO: buy signals become if prediction > current by some delta (~5%). Reverse is sell (decrease by some delta). Senstitvity analysis should be conducted to compare this delta level
#TODO: Use quantstats for a HTMl tearsheet
#TODO: market-regime detector with Hiden-markov model
#TODO: Add a Echo State Networks (ESN) layer to the model
#TODO: randomly sample 50 tickers, run backtest for all of them, and plot. take average sharpe ratio, and other metrics

In [ ]:
# Set the Wikipedia page title and section header
tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
# Clean up the dataframe
tickers = tickers.iloc[:, [1]].to_numpy().flatten()

In [ ]:
#tickers = ['IONQ','QBTS','RGTI']
training_dfs = []
stocks_dfs = []
for ticker in tickers:
    training_data, raw_stock_data = TickerData(ticker,years=10).process_all()
    training_dfs.append(training_data)
    stocks_dfs.append(raw_stock_data)

training_data = pd.concat(training_dfs, ignore_index=False)
stock_data = pd.concat(stocks_dfs, ignore_index=False)

In [ ]:
#training_data.to_csv("Data/NASDAQ_100_TrainingData_v2.csv", index=True)
#stock_data.to_csv("Data/NASDAQ_100_StockData_v2.csv", index=True)

In [2]:
training_data = pd.read_csv("Data/NASDAQ_100_TrainingData_v2.csv")
training_data = training_data.set_index(training_data['Date']).drop(columns=['Date'])

In [4]:
stock_data = pd.read_csv("Data/NASDAQ_100_StockData_v2.csv")
stock_data = stock_data.set_index(stock_data['Date']).drop(columns=['Date'])

In [9]:
# --- Feature Extraction ---
# Calculate daily log returns and the daily high-low range
stock_data['Return'] = stock_data["Close"].pct_change()
stock_data['Range'] = stock_data['High'] - stock_data['Low']

# Calculate volatility as the rolling standard deviation of Close prices (7-day window)
stock_data['Volatility'] = stock_data['Close'].rolling(window=7).std()

# Drop the first days with NaN values (due to pct_change and rolling std operations)
stock_data.dropna(inplace=True)

# Prepare the feature matrix for HMM (e.g., returns, range, volume, and volatility as features)
features = stock_data[['Return', 'Range', 'Volume']].values  # shape (n_samples, n_features)
features

array([[-1.15743426e-02,  5.90003967e-01,  2.06980000e+06],
       [-2.66134392e-02,  1.77000427e+00,  3.85650000e+06],
       [ 1.05262697e-02,  8.80004883e-01,  1.68830000e+06],
       ...,
       [ 2.27582760e-02,  3.57998657e+00,  1.57380000e+06],
       [ 2.79220505e-02,  5.19000244e+00,  2.31160000e+06],
       [-1.93760384e-02,  6.36999512e+00,  1.67620000e+06]])

In [27]:
from hmmlearn import hmm

# --- HMM Model Training Tuning ---
n_components = 3  # number of hidden states (regimes)
model = hmm.GaussianHMM(n_components=n_components, covariance_type="full", n_iter=100, random_state=5)
model.fit(features)  # train the HMM on our features

converged = model.monitor_.converged
log_likelihood = model.score(features)
bic = model.bic(features)
aic = model.aic(features)
log_likelihood

-4032043.2245185673

In [12]:
# --- Decode hidden states for each day ---
hidden_states = model.predict(features)
stock_data['State'] = hidden_states  # add the numeric state to the DataFrame for reference

# Examine the learned state means (for the Return and Range features)
for i in range(n_components):
    mean_return = model.means_[i, 0]
    vol_range = model.means_[i, 1]
    print(f"State {i}: mean return = {mean_return:.4f}, mean range = {vol_range:.4f}")

State 0: mean return = 0.0007, mean range = 9.1711
State 1: mean return = 0.0009, mean range = 1.1427
State 2: mean return = 0.0007, mean range = 4.0468


In [13]:
# --- Label each hidden state as Bullish, Bearish, or Neutral ---
# Determine ordering of states by mean return (the first feature in model.means_)
state_order = np.argsort(model.means_[:, 0])  # indices of states sorted by mean return
# Map state index to regime name
state_labels = {}
state_labels[state_order[0]] = "Bearish"   # lowest mean return
state_labels[state_order[1]] = "Neutral"   # middle
state_labels[state_order[2]] = "Bullish"   # highest mean return

# Create a new column with the regime label for each day
stock_data['Regime'] = stock_data['State'].map(state_labels)
stock_data[['State', 'Ticker', 'Regime']] # show first 10 days with numeric state and label


,State,Ticker,Regime
Date,,,
2015-04-16 00:00:00-04:00,1,ADBE,Bullish
2015-04-17 00:00:00-04:00,1,ADBE,Bullish
2015-04-20 00:00:00-04:00,1,ADBE,Bullish
2015-04-21 00:00:00-04:00,1,ADBE,Bullish
2015-04-22 00:00:00-04:00,1,ADBE,Bullish
...,...,...,...
2025-03-20 00:00:00-04:00,0,ZS,Bearish
2025-03-21 00:00:00-04:00,0,ZS,Bearish
2025-03-24 00:00:00-04:00,0,ZS,Bearish


In [24]:
# --- Visualization of regimes ---
ticker = np.random.choice(stock_data['Ticker'].unique())
fig_data = stock_data[stock_data['Ticker'] == ticker] # Randomly sample a ticker

# Define colors for each regime
fig = make_subplots(rows=3, cols=1, subplot_titles=('Bearish Hidden State', 'Neutral Hidden State', 'Bullish Hidden State'))

bearish_mask = fig_data['Regime'] == "Bearish"
fig.add_trace(go.Scatter(x=fig_data.index[bearish_mask], y=fig_data['Close'][bearish_mask], mode="markers",name="Bearish", line=dict(color="red")),row=1,col=1)
neutral_mask = fig_data['Regime'] == "Neutral"
fig.add_trace(go.Scatter(x=fig_data.index[neutral_mask], y=fig_data['Close'][neutral_mask], mode="markers",name="Neutral", line=dict(color="orange")),row=2,col=1)
bullish_mask = fig_data['Regime'] == "Bullish"
fig.add_trace(go.Scatter(x=fig_data.index[bullish_mask], y=fig_data['Close'][bullish_mask], mode="markers",name="Bullish", line=dict(color="green")),row=3,col=1)

fig.update_layout(
    title=f'Market Regimes Identified by HMM for {ticker}',
    xaxis_title="Date",
    height=800,
    yaxis_title="Stock Price (USD)",
    template='plotly_white',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

In [28]:
import pickle
# --- Exporting (Saving) the model ---
with open("Models/hmm_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved successfully.")

# Later, or in another script, you can import the model as follows:
with open("Models/hmm_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

# Now you can use loaded_model to predict on new data.
# Just ensure that the new dataset is preprocessed in the same way as the training data.
new_hidden_states = loaded_model.predict(features)  # where new_features is the feature matrix of your new dataset


Model saved successfully.


In [ ]:
data_module = DataModule(training_data, window_size=50, batch_size=32)

In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import torch.nn.functional as F
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import shap

class TEMPUS(nn.Module):
    """
    The TEMPUS model for time-series data processing and prediction tasks.

    TEMPUS implements a hybrid deep learning architecture
    that combines LSTM at multiple temporal resolutions, Temporal Convolution Networks (TCN), feature fusion, Temporal Attention (TA), and fully connected layers for regression tasks. It is designed
    to handle sequential data effectively by capturing temporal
    relationships and high-level data representations.

    """
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2,tcn_kernel_sizes=[3, 5, 7], device='cpu'):
        super(TEMPUS, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.dropout_rate = dropout

        # Multiple Temporal Resolutions of LSTM
        self.lstm_short = nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_size,num_layers=self.num_layers,batch_first=True, dropout=self.dropout_rate if self.num_layers > 1 else 0,bidirectional=True)
        self.lstm_medium = nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_size,num_layers=self.num_layers,batch_first=True,dropout=self.dropout_rate if self.num_layers > 1 else 0,bidirectional=True)
        self.temporal_fusion = nn.Linear(hidden_size * 4, hidden_size * 2) # Fusion layer for temporal resolutions

        # Temporal Convolutional Network (TCN)
        self.tcn_modules = nn.ModuleList()
        for k_size in tcn_kernel_sizes:
            padding = (k_size - 1) // 2  # Same padding
            self.tcn_modules.append(nn.Sequential(
                nn.Conv1d(input_size, hidden_size, kernel_size=k_size, padding=padding),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Conv1d(hidden_size, hidden_size, kernel_size=k_size, padding=padding),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU()
            ))
        self.tcn_fusion = nn.Linear(hidden_size * len(tcn_kernel_sizes), hidden_size * 2)

        # Combine TCN and LSTM features
        self.feature_fusion = nn.Linear(hidden_size * 4, hidden_size * 2)

        # Temporal attention
        self.temporal_attention = TemporalAttention(
            d_model=hidden_size * 2,
            nhead=4,
            dropout=dropout
        )

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.regression_head = nn.Linear(hidden_size // 2, 1)
        self.dropout = nn.Dropout(dropout)

    def downsample_sequence(self, x, factor):
        """Downsample time sequence by average pooling"""
        batch_size, seq_len, features = x.size()
        if seq_len % factor != 0:
            # Pad sequence if needed
            pad_len = factor - (seq_len % factor)
            x = F.pad(x, (0, 0, 0, pad_len))
            seq_len += pad_len

        # Reshape for pooling
        x = x.view(batch_size, seq_len // factor, factor, features)
        # Average pool
        x = torch.mean(x, dim=2)
        return x

    def forward(self, x):
        batch_size, seq_len, features = x.size()

        # Process with TCN
        tcn_outputs = []
        x_tcn = x.transpose(1, 2)  # TCN expects (batch, channels, seq_len)
        for tcn_module in self.tcn_modules:
            tcn_out = tcn_module(x_tcn)
            tcn_outputs.append(tcn_out)

        # Concatenate TCN outputs
        tcn_combined = torch.cat(tcn_outputs, dim=1)
        tcn_combined = tcn_combined.transpose(1, 2)  # Back to (batch, seq, features)
        tcn_features = self.tcn_fusion(tcn_combined)

        # Multiple Temporal Resolutions
        # Original sequence for short-term patterns
        lstm_short_out, _ = self.lstm_short(x)

        # Downsampled sequence for medium-term patterns (every 2 time steps)
        x_medium = self.downsample_sequence(x, 2)
        lstm_medium_out, _ = self.lstm_medium(x_medium)

        # Upsample medium resolution back to original sequence length
        lstm_medium_out = F.interpolate(
            lstm_medium_out.transpose(1, 2),
            size=seq_len,
            mode='linear'
        ).transpose(1, 2)

        # Combine temporal resolutions
        lstm_combined = torch.cat([lstm_short_out, lstm_medium_out], dim=2)
        lstm_features = self.temporal_fusion(lstm_combined)

        # 2. Add residual connection
        if features == lstm_features.size(2):  # If dimensions match
            lstm_features = lstm_features + x

        # Combine LSTM and TCN features
        combined_features = torch.cat([lstm_features, tcn_features], dim=2)
        fused_features = self.feature_fusion(combined_features)

        # Apply temporal attention
        attended_features = self.temporal_attention(fused_features)

        # Global pooling (average) across time dimension to get single feature vector per sequence
        pooled_features = torch.mean(attended_features, dim=1)

        # Final output layers
        x = F.relu(self.fc1(pooled_features))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        outputs = self.regression_head(x)

        return outputs

    def train_model(self, train_loader, test_loader, criterion, optimizer, num_epochs=10, clip_value=1.0):
        """
        Train the model with a regression task
        """
        self.to(self.device)

        best_test_mape = float('inf')
        best_model_state = None

        history = {
            'train_loss': [], 'test_loss': [],
            'rmse': [], 'mape': []
        }

        epoch_progress = tqdm(range(num_epochs), desc="Training Epochs")
        for epoch in epoch_progress:
            self.train()
            running_loss = 0.0

            for inputs, targets in train_loader:
                inputs = inputs.to(self.device)
                outputs = self(inputs).squeeze()
                loss = criterion(outputs, targets)

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(self.parameters(), max_norm=clip_value)
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Evaluate on validation set
            test_loss, test_rmse, test_mape = self.evaluate(test_loader, criterion)
            epoch_progress.set_postfix({
                'Train Loss': f'{train_loss:.4f}',
                'Test Loss': f'{test_loss:.4f}',
                'RMSE': f'{test_rmse:.4f}',
                'MAPE': f'{test_mape:.2f}%'
            })
            #print(f"Epoch {epoch}/{num_epochs}; Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, RMSE: {test_rmse:.2f}%")

            # Store history
            history['train_loss'].append(train_loss)
            history['test_loss'].append(test_loss)
            history['rmse'].append(test_rmse)
            history['mape'].append(test_mape)

            # Store model state with best val mape
            if test_mape < best_test_mape:
                best_test_mape = test_mape
                best_model_state = self.state_dict()

        # Load the best model state before returning
        if best_model_state is not None:
            self.load_state_dict(best_model_state)

        # Final evaluation with best model state
        test_loss, test_rmse, test_mape = self.evaluate(test_loader, criterion)
        print(f"\nBest Loss: {test_loss:.4f}, Best RMSE: {test_rmse:.4f}, Best MAPE: {test_mape:.2f}%")

        self.history = history

        return history

    def evaluate(self, data_loader, criterion):
        self.to(self.device)
        self.eval()
        total_loss = 0
        all_predictions = []
        all_targets = []

        with torch.no_grad():
            for inputs, targets in data_loader:
                inputs = inputs.to(self.device)
                outputs = self(inputs).squeeze()
                loss = criterion(outputs, targets)

                total_loss += loss.item() * inputs.size(0)

                all_predictions.append(outputs.to(self.device).numpy())
                all_targets.append(targets.to(self.device).numpy())

        all_predictions = np.concatenate(all_predictions, axis=0)
        all_targets = np.concatenate(all_targets, axis=0)

        # Calculate RMSE and MAPE
        mse = np.mean((all_predictions - all_targets) ** 2)
        rmse = np.sqrt(mse)
        # Avoid division by zero by adding a small epsilon
        mape = np.mean(np.abs((all_targets - all_predictions) / (all_targets + 1e-8))) * 100
        avg_loss = total_loss / len(data_loader.dataset)

        return avg_loss, rmse, mape

In [ ]:
class TCNBlock(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, dilation, padding, dropout=0.2):
        super(TCNBlock, self).__init__()

        self.conv1 = nn.Conv1d(
            in_channels=input_dim,
            out_channels=output_dim,
            kernel_size=kernel_size,
            dilation=dilation,
            padding=padding
        )
        self.norm1 = nn.BatchNorm1d(output_dim)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(
            in_channels=output_dim,
            out_channels=output_dim,
            kernel_size=kernel_size,
            dilation=dilation,
            padding=padding
        )
        self.norm2 = nn.BatchNorm1d(output_dim)
        self.relu2 = nn.ReLU()  # Added missing relu2 activation
        self.dropout2 = nn.Dropout(dropout)

        # Residual connection if dimensions don't match
        self.residual = nn.Conv1d(input_dim, output_dim, 1) if input_dim != output_dim else nn.Identity()

    def forward(self, x):

        # First conv block
        # Residual input
        residual = self.residual(x)

        # First conv block
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu1(out)
        out = self.dropout1(out)

        # Second conv block
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.relu2(out)
        out = self.relu2(out)  # Correctly use relu2
        out = self.dropout2(out)

        # Return to original shape
        # Add the residual and pass through final activation
        return self.relu1(out + residual)  # Fixed to use relu1 for the final activation

In [ ]:
class EchoStateNetwork(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, spectral_radius=0.9,
                 sparsity=0.1, noise=0.001, bidirectional=False):
        super(EchoStateNetwork, self).__init__()

        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.noise = noise
        self.bidirectional = bidirectional

        # Input weights (fixed)
        self.register_buffer('W_in', self._initialize_input_weights())

        # Reservoir weights (fixed)
        self.register_buffer('W', self._initialize_reservoir_weights())

        # Output weights (trainable)
        self.W_out = nn.Linear(reservoir_size, output_size)

        if bidirectional:
            # Second set of weights for backward direction
            self.register_buffer('W_in_reverse', self._initialize_input_weights())
            self.register_buffer('W_reverse', self._initialize_reservoir_weights())
            self.W_out_reverse = nn.Linear(reservoir_size, output_size)
            # Combined output
            self.W_combined = nn.Linear(output_size * 2, output_size)

    def _initialize_input_weights(self):
        W_in = torch.zeros(self.reservoir_size, self.input_size)
        W_in = torch.nn.init.xavier_uniform_(W_in)
        return W_in

    def _initialize_reservoir_weights(self):
        # Create sparse matrix
        W = torch.zeros(self.reservoir_size, self.reservoir_size)
        num_connections = int(self.sparsity * self.reservoir_size * self.reservoir_size)
        indices = torch.randperm(self.reservoir_size * self.reservoir_size)[:num_connections]
        rows = indices // self.reservoir_size
        cols = indices % self.reservoir_size
        values = torch.randn(num_connections)
        W[rows, cols] = values

        # Scale to desired spectral radius
        eigenvalues = torch.linalg.eigvals(W)
        max_eigenvalue = torch.max(torch.abs(eigenvalues))
        W = W * (self.spectral_radius / max_eigenvalue)
        return W

    def _reservoir_step(self, x, h_prev, W_in, W):
        """Execute one step of the reservoir"""
        # h_new = tanh(W_in @ x + W @ h_prev + noise)
        h_new = torch.tanh(torch.mm(x, W_in.t()) + torch.mm(h_prev, W.t()) +
                           self.noise * torch.randn(h_prev.shape, device=h_prev.device))
        return h_new

    def forward(self, x):
        """
        x: input tensor of shape (batch_size, seq_len, input_size)
        """
        batch_size, seq_len, _ = x.size()

        # Forward pass
        h = torch.zeros(batch_size, self.reservoir_size, device=x.device)
        outputs_forward = []

        for t in range(seq_len):
            h = self._reservoir_step(x[:, t], h, self.W_in, self.W)
            outputs_forward.append(self.W_out(h))

        outputs_forward = torch.stack(outputs_forward, dim=1)  # (batch_size, seq_len, output_size)

        if not self.bidirectional:
            return outputs_forward

        # Backward pass for bidirectional ESN
        h_reverse = torch.zeros(batch_size, self.reservoir_size, device=x.device)
        outputs_reverse = []

        for t in range(seq_len - 1, -1, -1):
            h_reverse = self._reservoir_step(x[:, t], h_reverse, self.W_in_reverse, self.W_reverse)
            outputs_reverse.insert(0, self.W_out_reverse(h_reverse))

        outputs_reverse = torch.stack(outputs_reverse, dim=1)  # (batch_size, seq_len, output_size)

        # Combine forward and backward outputs
        combined = torch.cat((outputs_forward, outputs_reverse), dim=2)
        return self.W_combined(combined)

In [ ]:
class OldTemporalAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.time_attn = nn.Sequential(
            nn.Linear(1, 16),  # Simple time feature processing
            nn.Tanh(),
            nn.Linear(16, 1)
        )
        self.feature_attn = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_output, time_features):
        # lstm_output: [batch, seq_len, hidden]
        # time_features: [batch, seq_len, 1] - normalized position in sequence

        # Compute base attention scores from features
        feature_scores = self.feature_attn(lstm_output)  # [batch, seq_len, 1]
        # Compute time-based attention
        time_weights = self.time_attn(time_features)  # [batch, seq_len, 1]
        # Combine feature and time attention
        combined_scores = feature_scores + time_weights
        # Apply softmax to get attention weights
        attention_weights = torch.softmax(combined_scores, dim=1)
        # Apply attention to get context vector
        context = torch.bmm(attention_weights.transpose(1, 2), lstm_output)  # [batch, 1, hidden]

        return context, attention_weights

In [ ]:
class TemporalAttention(nn.Module):
    def __init__(self, d_model, nhead=4, dropout=0.1):
        super(TemporalAttention, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x shape: [batch_size, seq_len, d_model]
        attn_output, _ = self.self_attn(x, x, x)
        out = self.norm(x + self.dropout(attn_output))
        return out

In [ ]:
# Automatically get the number of features given my data_module object
input_size = data_module.num_features
hidden_size = 64
num_layers = 1
dropout = 0.2
epochs = 20

# Instantiate the model
model = TEMPUS(input_size, hidden_size, num_layers, dropout)
# Set up loss and optimizer
criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
# Train Model
history = model.train_model(data_module.train_loader, data_module.test_loader, criterion, optimizer, epochs)

In [ ]:
model.save_model("Models/TEMPUS_NASDAQ_100_v2.pt")
preds_def = model.get_predictions(training_data)
preds_def

In [ ]:
# Plot training metrics
training_fig = model.plot_training_history(training_data)
training_fig.show()

In [ ]:
# Get predictions
preds_df = model.get_predictions(training_data)
merged_df = pd.merge(stock_data, preds_df, on=['Date', 'Ticker'], how='inner')

In [ ]:
# Create a combined plot with stock prices and prediction markers
def plot_combined_predictions(data, ticker):
    # Filter for a particular ticker
    if type(ticker) == str:
        data = data[data['Ticker'] == ticker]
    else:
        return "Ticker provided is not a valid value"

    # Create figure
    fig = go.Figure()

    # Plot stock price trend line
    fig.add_trace(go.Scatter(
        x=data['Date'],
        y=data['Close'],
        mode='lines',
        name='Stock Price',
        line=dict(width=1)
    ))

    # Split signals by type and correctness
    buy_signals = data[data['Predicted'] == 2]
    sell_signals = data[data['Predicted'] == 1]
    hold_signals = data[data['Predicted'] == 0]

    # Correct/incorrect buy signals
    correct_buy = buy_signals[buy_signals['Predicted'] == buy_signals['Actual']]
    incorrect_buy = buy_signals[buy_signals['Predicted'] != buy_signals['Actual']]

    # Correct/incorrect sell signals
    correct_sell = sell_signals[sell_signals['Predicted'] == sell_signals['Actual']]
    incorrect_sell = sell_signals[sell_signals['Predicted'] != sell_signals['Actual']]

    # Correct/incorrect hold signals
    correct_hold = hold_signals[hold_signals['Predicted'] == hold_signals['Actual']]
    incorrect_hold = hold_signals[hold_signals['Predicted'] != hold_signals['Actual']]

    # Plot buy signals
    fig.add_trace(go.Scatter(
        x=correct_buy['Date'],
        y=data.loc[correct_buy.index]['Close'],
        mode='markers',
        name='Correct Buy Signal',
        marker=dict(symbol='triangle-up', size=10, color='green')
    ))

    fig.add_trace(go.Scatter(
        x=incorrect_buy['Date'],
        y=data.loc[incorrect_buy.index]['Close'],
        mode='markers',
        name='Incorrect Buy Signal',
        marker=dict(symbol='triangle-up', size=8, color='gray', opacity=0.2)
    ))

    # Plot sell signals
    fig.add_trace(go.Scatter(
        x=correct_sell['Date'],
        y=data.loc[correct_sell.index]['Close'],
        mode='markers',
        name='Correct Sell Signal',
        marker=dict(symbol='triangle-down', size=10, color='red')
    ))

    fig.add_trace(go.Scatter(
        x=incorrect_sell['Date'],
        y=data.loc[incorrect_sell.index]['Close'],
        mode='markers',
        name='Incorrect Sell Signal',
        marker=dict(symbol='triangle-down', size=8, color='gray', opacity=0.2)
    ))

    # Plot hold signals (using a different symbol)
    fig.add_trace(go.Scatter(
        x=correct_hold['Date'],
        y=data.loc[correct_hold.index]['Close'],
        mode='markers',
        name='Correct Hold Signal',
        marker=dict(symbol='circle', size=8, color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=incorrect_hold['Date'],
        y=data.loc[incorrect_hold.index]['Close'],
        mode='markers',
        name='Incorrect Hold Signal',
        marker=dict(symbol='circle', size=6, color='gray', opacity=0.2)
    ))

    # Update layout
    fig.update_layout(
        title=f'{ticker} Stock Price - Actual/Predicted Signals',
        #xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly_dark',
        height=600,
        legend=dict(orientation="h", yanchor="bottom", y=1.02)
    )

    fig.show()

# Call the modified function
plot_combined_predictions(merged_df, 'PLTR')

In [ ]:
from Components.BackTesting import BackTesting
import pandas as pd

#merged_df = pd.read_csv('Data/NASDAQ_100_PredictictionsData.csv')

initial_capital = 10000.0
ticker = 'PLTR'
backtester = BackTesting(merged_df, ticker, initial_capital)
results, _ = backtester.run_simulation()
trades_fig, value_fig, exposure_fig = backtester.plot_performance()
trades_fig.show()
value_fig.show()
exposure_fig.show()

In [ ]:
# %%
import ray
from ray import tune
from ray.air import session
#from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from Components.TrainModel import TunableLSTMClassifier
import os
from functools import partial


# Define a training function for Ray Tune
def train_lstm(config, input_size=33, num_classes=3, train_data=None, val_data=None, test_data=None):
    # Set up device
    device = "mps" if torch.backends.mps.is_available() else "cpu"

    # Create model with the hyperparameter configuration
    model = TunableLSTMClassifier({
        "input_size": input_size,
        "hidden_size": config["hidden_size"],
        "num_layers": config["num_layers"],
        "num_classes": num_classes,
        "dropout_rate": config["dropout_rate"]
    }).to(device)

    # Set up data loaders
    data_module = DataModule(train_data, seq_length=10, batch_size=config["batch_size"])
    train_loader = data_module.train_loader
    val_loader = data_module.eval_loader
    test_loader = data_module.test_loader

    # Set up loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(
        model.parameters(),
        lr=config["lr"],
        weight_decay=config["weight_decay"]
    )

    # Training loop
    for epoch in range(10):  # Limit epochs for tuning
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()

        val_accuracy = val_correct / val_total

        # Report metrics to Ray Tune
        session.report({
            "val_accuracy": val_accuracy,
            "val_loss": val_loss / len(val_loader),
            "train_accuracy": train_correct / train_total,
            "train_loss": train_loss / len(train_loader),
            "epoch": epoch
        })


In [ ]:
# %%
# Initialize Ray
ray.init()

# Define the hyperparameter search space
config = {
    "lr": tune.loguniform(1e-5, 1e-2),
    "hidden_size": tune.choice([32, 64, 128, 256]),
    "num_layers": tune.choice([1, 2, 3]),
    "dropout_rate": tune.uniform(0.1, 0.5),
    "weight_decay": tune.loguniform(1e-6, 1e-3),
    "batch_size": tune.choice([16, 32, 64, 128])
}

# Configure the ASHA scheduler
scheduler = ASHAScheduler(
    max_t=10,  # Maximum number of epochs
    grace_period=1,
    reduction_factor=2
)

# Set up the tuner
tuner = tune.Tuner(
    tune.with_resources(
        partial(
            train_lstm,
            input_size=33,
            num_classes=3,
            train_data=training_data,
            val_data=None,
            test_data=None
        ),
        resources={"cpu": 2, "gpu": 0}  # Adjust based on your hardware
    ),
    tune_config=tune.TuneConfig(
        metric="val_accuracy",
        mode="max",
        scheduler=scheduler,
        num_samples=50,  # Number of hyperparameter combinations to try
        trial_dirname_creator=lambda trial: f"{trial.trainable_name}_{trial.trial_id[:4]}"
    ),
    param_space=config
)

# Run the hyperparameter search
results = tuner.fit()


In [ ]:
# %%
# Get the best hyperparameters
best_result = results.get_best_result("val_accuracy", "max")
best_config = best_result.config
print("Best config:", best_config)

# Extract the best hyperparameters
best_lr = best_config["lr"]
best_hidden_size = best_config["hidden_size"]
best_num_layers = best_config["num_layers"]
best_dropout = best_config["dropout_rate"]
best_weight_decay = best_config["weight_decay"]
best_batch_size = best_config["batch_size"]

# Plot results
df_results = results.get_dataframe()

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))

# Plot learning rate vs validation accuracy
plt.subplot(2, 3, 1)
plt.scatter(df_results["config/lr"], df_results["val_accuracy"])
plt.xscale("log")
plt.xlabel("Learning Rate")
plt.ylabel("Validation Accuracy")

# Plot hidden size vs validation accuracy
plt.subplot(2, 3, 2)
plt.scatter(df_results["config/hidden_size"], df_results["val_accuracy"])
plt.xlabel("Hidden Size")
plt.ylabel("Validation Accuracy")

# Plot num_layers vs validation accuracy
plt.subplot(2, 3, 3)
plt.scatter(df_results["config/num_layers"], df_results["val_accuracy"])
plt.xlabel("Number of Layers")
plt.ylabel("Validation Accuracy")

# Plot dropout vs validation accuracy
plt.subplot(2, 3, 4)
plt.scatter(df_results["config/dropout_rate"], df_results["val_accuracy"])
plt.xlabel("Dropout Rate")
plt.ylabel("Validation Accuracy")

# Plot weight decay vs validation accuracy
plt.subplot(2, 3, 5)
plt.scatter(df_results["config/weight_decay"], df_results["val_accuracy"])
plt.xscale("log")
plt.xlabel("Weight Decay")
plt.ylabel("Validation Accuracy")

# Plot batch size vs validation accuracy
plt.subplot(2, 3, 6)
plt.scatter(df_results["config/batch_size"], df_results["val_accuracy"])
plt.xlabel("Batch Size")
plt.ylabel("Validation Accuracy")

plt.tight_layout()
plt.show()


In [ ]:
#ticker.get_balance_sheet(freq='quarterly')
#ticker.get_calendar()
#ticker.get_cash_flow(freq='quarterly')
#earnings_data = ticker.get_earnings_dates()
#income_statement = ticker.get_income_stmt(freq='yearly').T
#ticker.get_institutional_holders()
#ticker.get_recommendations()
#ticker.get_sustainability()

In [ ]:
# define a function to fetch the options data for a given ticker symbol
#def fetch_options_data(ticker_symbol):
    #ticker = yf.Ticker(ticker_symbol)
#    options_dates = ticker.options
#    options_data = ticker.option_chain(date='2025-03-21')
#    return options_data.calls, options_data.puts
##ionq_stock_data = ionq_stock_data.sort_values(by='Date', ascending=False)